In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random

# Определение нейронной сети для аппроксимации Q-функции
class QNetwork(nn.Module):
    def __init__(self, state_size, action_size):
        super(QNetwork, self).__init__()
        # Простая сеть с одним скрытым слоем
        self.fc1 = nn.Linear(state_size, 24)
        self.fc2 = nn.Linear(24, action_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Среда (упрощенная для примера)
class SimpleEnv:
    def __init__(self):
        self.state_space_size = 4
        self.action_space_size = 2

    def step(self, state, action):
        next_state = (state + 1) % self.state_space_size
        reward = 1 if action == 0 else -1
        done = next_state == 0
        return next_state, reward, done

    def reset(self):
        return 0

def train(env, model, episodes=1000, learning_rate=0.01, gamma=0.95):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_fn = nn.MSELoss()

    for episode in range(episodes):
        state = env.reset()
        done = False

        while not done:
            state_tensor = torch.FloatTensor([state])
            q_values = model(state_tensor)

            # Исправление: Изменение выбора действия
            _, action = torch.max(q_values, dim=0)
            action = action.item()

            next_state, reward, done = env.step(state, action)

            next_state_tensor = torch.FloatTensor([next_state])
            next_q_values = model(next_state_tensor)

            target_q = reward + gamma * torch.max(next_q_values)
            loss = loss_fn(q_values[action], target_q)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            state = next_state

    print(f"Обучение завершено после {episodes} эпизодов")

# Создание сети и обучение
state_size = 1  # Размерность состояния
action_size = 2  # Количество возможных действий

def random_search(env, state_size, action_size, num_iterations=10):
    best_reward = float('-inf')
    best_learning_rate = None
    best_gamma = None
    
    for _ in range(num_iterations):
        learning_rate = random.uniform(0.001, 0.1)
        gamma = random.uniform(0.9, 0.99)
        
        q_network = QNetwork(state_size, action_size)
        train(env, q_network, learning_rate=learning_rate, gamma=gamma)
        
        total_reward = test_model(env, q_network)
        
        if total_reward > best_reward:
            best_reward = total_reward
            best_learning_rate = learning_rate
            best_gamma = gamma
    
    print("Лучшие гиперпараметры:")
    print(f"Learning Rate: {best_learning_rate}")
    print(f"Gamma: {best_gamma}")
    print(f"Лучшая награда: {best_reward}")

def test_model(env, model, num_episodes=10):
    total_rewards = []
    for episode in range(num_episodes):
        state = env.reset()
        done = False
        total_reward = 0

        while not done:
            state_tensor = torch.FloatTensor([state])
            q_values = model(state_tensor)
            _, action = torch.max(q_values, dim=0)
            action = action.item()

            next_state, reward, done = env.step(state, action)
            total_reward += reward
            state = next_state

        total_rewards.append(total_reward)

    avg_reward = sum(total_rewards) / num_episodes
    print(f"Средняя награда на {num_episodes} эпизодах: {avg_reward}")
    return avg_reward

# Создание среды и запуск случайного поиска гиперпараметров
env = SimpleEnv()
random_search(env, state_size, action_size)


Обучение завершено после 1000 эпизодов
Средняя награда на 10 эпизодах: 4.0
Обучение завершено после 1000 эпизодов
Средняя награда на 10 эпизодах: 4.0
Обучение завершено после 1000 эпизодов
Средняя награда на 10 эпизодах: 4.0
Обучение завершено после 1000 эпизодов
Средняя награда на 10 эпизодах: 4.0
Обучение завершено после 1000 эпизодов
Средняя награда на 10 эпизодах: 4.0
Обучение завершено после 1000 эпизодов
Средняя награда на 10 эпизодах: 4.0
Обучение завершено после 1000 эпизодов
Средняя награда на 10 эпизодах: 4.0
Обучение завершено после 1000 эпизодов
Средняя награда на 10 эпизодах: 4.0
Обучение завершено после 1000 эпизодов
Средняя награда на 10 эпизодах: 4.0
Обучение завершено после 1000 эпизодов
Средняя награда на 10 эпизодах: 4.0
Лучшие гиперпараметры:
Learning Rate: 0.09338708123463178
Gamma: 0.9244641684212151
Лучшая награда: 4.0
